In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time
from sklearn import metrics
import requests
import os
import yaml
import json
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
from sklearn.metrics import confusion_matrix
import requests, pprint
import uuid

## Lab 2 - Continuous Learning

### Step 1: Load configs

In [3]:
%run ..\Utils\utils.py
%run ..\Utils\utils-cv.py

In [26]:
config = load_configs(os.path.join("Config", "config.yaml"))

train_endpoint = config['cv']['TrainingEndpoint']
training_key = config['cv']['TrainingKey']
pred_endpoint = config['cv']['PredictionEndpoint']
prediction_key = config['cv']['PredictionKey']
prediction_resource_id = "/subscriptions/8d47f388-3a7a-4970-9dec-2c6cc53e02a9/resourceGroups/rg_chihuahua_muffin/providers/Microsoft.CognitiveServices/accounts/cmcustomvision-prediction"
project_name = config['cv']['ProjectName']
project_id = config['cv']['ProjectID']
threshold = config['cv']['Threshold']
path_training_images = os.path.join(get_project_dir(), "Deploy", "training-images")


Authenticate the client

In [5]:
cv = CustomVisionProject()
trainer = cv.get_trainer(training_key, train_endpoint)
predictor = cv.get_predictor(prediction_key, pred_endpoint)

In [6]:
##
cv.get_project(project_id)
print(cv.project.id)

8217f4df-bf35-4e9a-8dfc-6a5e18b4d1a5


In [7]:
cv.project.name

'fruit-classifier-gerdau'

### Step 2: Getting images received by the endpoint

In the Custon Vision portal, is the images from **Predictions** tab.

In [36]:
# Printar as tags que o modelo previu através do endpoint 
#limite de duas queries por vez
tags_ids = [t['id'] for t in trainer.get_tags(project_id, raw=True).response.json()]
queries = []
query_pred_bytags = {}
query_pred_bytags['tags'] = []
coringa = []

for i in tags_ids:
    tags = []
    tags.append({"id" : i, "minThreshold" : 0.1})
    queries.append({"tags" : tags } )


In [37]:
#creating query to get the images received by the endpoint
batch_img_tags = {}
batch_img_tags["images"] = []
for i in queries:
    predicted_imgs = trainer.query_predictions(project_id, query=i, raw=True).response.json()
    for r in predicted_imgs["results"]:
        batch_img_tags["images"].append({ "id" : r["id"], "tagId" : r["predictions"][0]["tagId"] })

In [38]:
print(json.dumps(batch_img_tags, indent=2))

{
  "images": [
    {
      "id": "61f2de46-e32e-49fb-85f7-e8481933ee51",
      "tagId": "c9d388dc-cd14-4b0c-b35e-858a84ce75db"
    },
    {
      "id": "a9e74e16-6075-4fa8-a521-264ab0dfab24",
      "tagId": "3bc16af9-9c12-4dea-846a-ad33206c3307"
    },
    {
      "id": "0c8b99f5-21ac-4a7c-bbcf-6caae71fafb9",
      "tagId": "8bcf4dc4-ae2b-47f5-9315-aecf4c88cc3a"
    },
    {
      "id": "3f26fc10-c7e1-4cc6-afeb-de62040aec96",
      "tagId": "8bcf4dc4-ae2b-47f5-9315-aecf4c88cc3a"
    }
  ]
}


### Step 3: Send the images from **Predictions** to **Training Images**


In [39]:
# send the images received by the endpoit to the Training Images
def create_image_from_predictions_rest(documents):
    ''' Endpoints: entities, sentiment, languages, keyPhrases'''

    #endpoint = train_endpoint + 'customvision/v3.3/training/projects/{}/images/tags'.format(project.id)
    endpoint = 'https://eastus.api.cognitive.microsoft.com' + '/customvision/v3.3/Training/projects/{}/images/predictions'.format(project_id)
    #print(project_id)
    #print(endpoint)
    # Setup the header information for the REST request passing in the subscription key
    headers = {"Training-key": training_key, "Content-Type" : "application/json"}
    #print(training_key)
    # Build the REST request by passing in the complete URL, header information for authentication, and the JSON document
    response = requests.post(endpoint, headers=headers, json=documents)

    # Create a variable to store the results that are returned from the REST request
    entities = response.json()

    # Output the result using pprint.
    #print(entities)


In [40]:
create_image_from_predictions_rest(batch_img_tags)

In [41]:
# get untagged images
print(len(cv.get_untagged_imgs()))

4


### Step 4: Filter only the images with the threshold above the expected and tagg them


In [42]:
threshold

0.8

In [43]:
#Filter just images with predictions more than the threshold defined to generate the query to tag them
untagged_img = [i.id  for i in trainer.get_untagged_images(cv.project.id)]

sugg =trainer.suggest_tags_and_regions(cv.project.id, cv.get_iteration_id(), untagged_img, raw = True).response.json()
batch_img_tags = {}
batch_img_tags["tags"] = []
for r in sugg:
    #print(r)
    if (r["predictions"][0]["probability"] > threshold):
        batch_img_tags["tags"].append({ "imageId" : r["id"], 
                                        "tagId" : r["predictions"][0]["tagId"]})
len(batch_img_tags["tags"])                                        

3

In [44]:
print(json.dumps(batch_img_tags, indent=2))

{
  "tags": [
    {
      "imageId": "a5bccf0f-deea-47eb-b893-e54a27b0a824",
      "tagId": "8bcf4dc4-ae2b-47f5-9315-aecf4c88cc3a"
    },
    {
      "imageId": "5e423a94-124f-496d-a2df-452a46fe154e",
      "tagId": "3bc16af9-9c12-4dea-846a-ad33206c3307"
    },
    {
      "imageId": "abd18af1-e585-430c-9c69-32d583132aac",
      "tagId": "c9d388dc-cd14-4b0c-b35e-858a84ce75db"
    }
  ]
}


In [45]:
# o metodo da sdk trainer.create_image_tags não está funcionando
import requests, pprint

def create_image_tags_rest(documents):
    ''' Endpoints: entities, sentiment, languages, keyPhrases'''

    #endpoint = train_endpoint + 'customvision/v3.3/training/projects/{}/images/tags'.format(project.id)
    endpoint = 'https://eastus.api.cognitive.microsoft.com' + '/customvision/v3.3/Training/projects/{}/images/tags'.format(cv.project.id)
    print(project_id)
    print(endpoint)
    # Setup the header information for the REST request passing in the subscription key
    headers = {"Training-key": training_key, "Content-Type" : "application/json"}
    print(training_key)
    # Build the REST request by passing in the complete URL, header information for authentication, and the JSON document
    response = requests.post(endpoint, headers=headers, json=documents)

    # Create a variable to store the results that are returned from the REST request
    entities = response.json()

    # Output the result using pprint.
    print(entities)


In [46]:
create_image_tags_rest(batch_img_tags)

8217f4df-bf35-4e9a-8dfc-6a5e18b4d1a5
https://eastus.api.cognitive.microsoft.com/customvision/v3.3/Training/projects/8217f4df-bf35-4e9a-8dfc-6a5e18b4d1a5/images/tags
8b5460f998a74816ad0f4d5c662defa7
{'created': [{'imageId': 'abd18af1-e585-430c-9c69-32d583132aac', 'tagId': 'c9d388dc-cd14-4b0c-b35e-858a84ce75db'}, {'imageId': '5e423a94-124f-496d-a2df-452a46fe154e', 'tagId': '3bc16af9-9c12-4dea-846a-ad33206c3307'}, {'imageId': 'a5bccf0f-deea-47eb-b893-e54a27b0a824', 'tagId': '8bcf4dc4-ae2b-47f5-9315-aecf4c88cc3a'}], 'duplicated': [], 'exceeded': []}


In [47]:
# get untagged images
print(len(cv.get_untagged_imgs()))

1


Now tag the leftover image at custom vision portal and upload at least 15 images (but remember the recommended is at least 50 per tag to ensure performance).

In [48]:
cv.upload_local_images(os.path.join(project_path(),'Deploy', 'new-tags'), ['caju'])

Uploading images...
Uploading 'caju' images:
Image 38494331000-9c35c44ce0-5k.jpg sucessfully uploaded!
Image 58bd0787ab596f4286730bacc710231e.jpg sucessfully uploaded!
Image caju-2-1200x738-1.jpg sucessfully uploaded!
Image caju-7.png sucessfully uploaded!
Image caju-thumb11400223.jpg sucessfully uploaded!
Image caju11-203829d000569330bb15897565181320-640-0.jpg sucessfully uploaded!
Image depositphotos_54896479-stock-photo-brazilian-caju-cashew-fruit.jpg sucessfully uploaded!
Image download.htm sucessfully uploaded!
Image D_NQ_NP_968734-MLB31610550130_072019-O.jpg sucessfully uploaded!
Image f131519cc9c5dd1e06c207237cab3b4f.jpg sucessfully uploaded!
Image fruta-decorativa-caju-frutas-decorativas.jpg sucessfully uploaded!
Image images.jfif sucessfully uploaded!
Image img1.jfif sucessfully uploaded!
Image img2.jfif sucessfully uploaded!
Image img3.jpg sucessfully uploaded!
Image img4.jpg sucessfully uploaded!


### Step 5: Re-train

In [49]:
cv.train_model()

Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Training ...
Completed ...
Model trained!
Iteration id: 5d580275-6166-41cf-bf5a-c1cfe0a9ec7e
Iteration name: Iteration 2


### Step 6: Publish the new iteration

In [50]:
publish_iteration_name = str(uuid.uuid1())
cv.trainer.publish_iteration(project_id, cv.get_iteration_id(), publish_iteration_name, prediction_resource_id)


True

In [51]:
# Classify test images
test_img_dir = os.path.join(project_path(),'Deploy','test-images')
image = 'IMG_TEST_4.JPG'

image_data = open(os.path.join(test_img_dir,image), "rb").read()
result = (cv.predictor.classify_image(project_id, cv.get_iteration_publis_name(), image_data))
print(image, ': {} ({:.0%})'.format(result.predictions[0].tag_name, result.predictions[0].probability))


IMG_TEST_4.JPG : caju (100%)


In [52]:
img_url = 'https://th.bing.com/th/id/OIP.1tq1RrlV1TnjuP4eNx9pcAHaE8?pid=ImgDet&rs=1'
result = (cv.predictor.classify_image_url(project_id, publish_iteration_name, img_url ))
print(img_url, ': {} ({:.0%})'.format(result.predictions[0].tag_name, result.predictions[0].probability))

https://th.bing.com/th/id/OIP.1tq1RrlV1TnjuP4eNx9pcAHaE8?pid=ImgDet&rs=1 : caju (98%)


In [53]:

img_url = 'https://th.bing.com/th/id/R.d21316cc72adbdc748c915094a063173?rik=H50nE3XeVuxm%2bQ&riu=http%3a%2f%2fstreetsmartbrazil.files.wordpress.com%2f2008%2f12%2fcaju.jpg&ehk=62O4h64%2bU43NDKdzUf8uMk1qZJJOufCoL%2byM9WhreYw%3d&risl=&pid=ImgRaw&r=0'
result = (cv.predictor.classify_image_url(project_id, publish_iteration_name, img_url ))
print(img_url, ': {} ({:.0%})'.format(result.predictions[0].tag_name, result.predictions[0].probability))

https://th.bing.com/th/id/R.d21316cc72adbdc748c915094a063173?rik=H50nE3XeVuxm%2bQ&riu=http%3a%2f%2fstreetsmartbrazil.files.wordpress.com%2f2008%2f12%2fcaju.jpg&ehk=62O4h64%2bU43NDKdzUf8uMk1qZJJOufCoL%2byM9WhreYw%3d&risl=&pid=ImgRaw&r=0 : caju (100%)
